In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(0.1)
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 16, 3)
        self.conv4_bn = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16, 32, 3)
        self.conv5_bn = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 32, 3)
        self.gap = nn.AvgPool2d((1,1))
        self.fc = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2_bn(self.dropout(self.conv2(F.relu(self.conv1_bn(self.dropout(self.conv1(x)))))))))
        # x = self.conv11(x)
        x = self.pool2(F.relu(self.conv4_bn(self.dropout(self.conv4(F.relu(self.conv3_bn(self.dropout(self.conv3(x)))))))))
        x = F.relu(self.conv6(F.relu(self.conv5_bn(self.dropout(self.conv5(x))))))
        x = self.gap(x)
        x = x.view(-1, 32)
        x = self.fc(x)
        return F.log_softmax(x)

In [19]:
model = Net().to(device)

In [20]:
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
           Dropout-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             584
           Dropout-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
         MaxPool2d-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 16, 12, 12]           1,168
           Dropout-9           [-1, 16, 12, 12]               0
      BatchNorm2d-10           [-1, 16, 12, 12]              32
           Conv2d-11           [-1, 16, 10, 10]           2,320
          Dropout-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        MaxPool2d-14             [-1, 1

<ipython-input-18-d75d3707dab5>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [21]:


torch.manual_seed(1),
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [22]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [23]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-18-d75d3707dab5>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.03664037957787514 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.87it/s]



Test set: Average loss: 0.0974, Accuracy: 9723/10000 (97.2%)



loss=0.041911233216524124 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.66it/s]



Test set: Average loss: 0.0808, Accuracy: 9760/10000 (97.6%)



loss=0.043923959136009216 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.93it/s]


...


Test set: Average loss: 0.0248, Accuracy: 9917/10000 (99.2%)



loss=0.07131240516901016 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.55it/s]



Test set: Average loss: 0.0201, Accuracy: 9934/10000 (99.3%)



loss=0.032812248915433884 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.75it/s]



Test set: Average loss: 0.0201, Accuracy: 9936/10000 (99.4%)

